# Extensión de la API de Pandas

In [2]:
import itertools
import pandas as pd
import upsetplot
from statsmodels.graphics.mosaicplot import mosaic

In [3]:
try:
    del pd.DataFrame.missing
except AttributeError:
    pass

In [4]:
'''
Referencias:
Para efectos practicos se hace uso indistinto e indiscriminado entre las siguientes palabras:
1) Casos == Observaciones == Filas
2) Variables == Columnas
'''

@pd.api.extensions.register_dataframe_accessor("missing")
class DontMissMe:
    def __init__(self, pandas_obj):
        self._obj = pandas_obj

    #metodo para obtener número total de valores faltante
    def number_missing(self) -> int:
        return self._obj.isna().sum().sum()

    #metodo para obtener el número total de valores completos (sin observaciones faltantes)
    def number_complete(self) -> int:
        return self._obj.size - self._obj.missing.number_missing()

    #metodo para obtener tabla con el conteo de datos faltantes en todas las columnas
    def missing_variable_summary(self) -> pd.DataFrame:
        return self._obj.isnull().pipe(
            lambda df_1: (
                df_1.sum()
                .reset_index(name="n_missing")
                .rename(columns={"index": "variable"})
                .assign(
                    n_cases=len(df_1),
                    pct_missing=lambda df_2: df_2.n_missing / df_2.n_cases * 100,
                )
            )
        )

    #metodo para obtener tabla con el conteo de datos faltantes en todas las filas
    def missing_case_summary(self) -> pd.DataFrame:
        return self._obj.assign(
            case=lambda df: df.index,
            n_missing=lambda df: df.apply(
                axis="columns", func=lambda row: row.isna().sum()
            ),
            pct_missing=lambda df: df["n_missing"] / df.shape[1] * 100,
        )[["case", "n_missing", "pct_missing"]]

    #metodo para obtener tabla agrupada por el conteo de datos faltantes en todas las columnas
    def missing_variable_table(self) -> pd.DataFrame:
        return (
            self._obj.missing.missing_variable_summary()
            .value_counts("n_missing")
            .reset_index()
            .rename(columns={"n_missing": "n_missing_in_variable", 0: "n_variables"})
            .assign(
                pct_variables=lambda df: df.n_variables / df.n_variables.sum() * 100
            )
            .sort_values("pct_variables", ascending=False)
        )

    #metodo para obtener tabla agrupada por el conteo de datos faltantes en todas las filas
    def missing_case_table(self) -> pd.DataFrame():
        return (
            self._obj.missing.missing_case_summary()
            .value_counts("n_missing")
            .reset_index()
            .rename(columns={"n_missing": "n_missing_in_case", 0: "n_cases"})
            .assign(pct_case=lambda df: df.n_cases / df.n_cases.sum() * 100)
            .sort_values("pct_case", ascending=False)
        )

    #metodo para obtener tabla agrupada por intervalos de filas para visualizar valores faltantes en una columna especifica
    def missing_variable_span(self, variable: str, span_every: int) -> pd.DataFrame:
        return (
            self._obj.assign(
                span_counter=lambda df: (
                    np.repeat(a=range(df.shape[0]), repeats=span_every)[: df.shape[0]]
                )
            )
            .groupby("span_counter")
            .aggregate(
                n_in_span=(variable, "size"),
                n_missing=(variable, lambda s: s.isnull().sum()),
            )
            .assign(
                n_complete=lambda df: df.n_in_span - df.n_missing,
                pct_missing=lambda df: df.n_missing / df.n_in_span * 100,
                pct_complete=lambda df: 100 - df.pct_missing,
            )
            .drop(columns=["n_in_span"])
            .reset_index()
        )

    #metodo para obtener tabla agrupada por racha de valores faltantes en una columna especifica
    def missing_variable_run(self, variable) -> pd.DataFrame:
        rle_list = self._obj[variable].pipe(
            lambda s: [[len(list(g)), k] for k, g in itertools.groupby(s.isnull())]
        )

        return pd.DataFrame(data=rle_list, columns=["run_length", "is_na"]).replace(
            {False: "complete", True: "missing"}
        )

    #metodo para obtener tabla ordenada por columnas de acuerdo a la cantidad de valores faltantes
    def sort_variables_by_missingness(self, ascending=False):

        return self._obj.pipe(
            lambda df: (df[df.isna().sum().sort_values(ascending=ascending).index])
        )

    #metodo para obtener tabla de matriz de sombras, only missing=True permite solo columnas con datos faltantes
    def create_shadow_matrix(
        self,
        true_string: str = "Missing",
        false_string: str = "Not Missing",
        only_missing: bool = False,
        suffix: str = "_NA",
    ) -> pd.DataFrame:
        return (
            self._obj.isna()
            .pipe(lambda df: df[df.columns[df.any()]] if only_missing else df)
            .replace({False: false_string, True: true_string})
            .add_suffix(suffix)
        )

    #metodo para obtener tabla con una matriz nabular(tabla original+matriz de sombras), only missing=True permite solo columnas con datos faltantes
    def bind_shadow_matrix(
        self,
        true_string: str = "Missing",
        false_string: str = "Not Missing",
        only_missing: bool = False,
        suffix: str = "_NA",
    ) -> pd.DataFrame:
        return pd.concat(
            objs=[
                self._obj,
                self._obj.missing.create_shadow_matrix(
                    true_string=true_string,
                    false_string=false_string,
                    only_missing=only_missing,
                    suffix=suffix,
                ),
            ],
            axis="columns",
        )

    def missing_scan_count(self, search) -> pd.DataFrame:
        return (
            self._obj.apply(axis="rows", func=lambda column: column.isin(search))
            .sum()
            .reset_index()
            .rename(columns={"index": "variable", 0: "n"})
            .assign(original_type=self._obj.dtypes.reset_index()[0])
        )
        
        
    #metodo para obtener relleno de columna con dummies, para usar en funcion de scatterplot
    def column_fill_with_dummies(
        self,
        column: pd.Series,
        proportion_below: float=0.10,
        jitter: float=0.075,
        seed: int=42
    ) -> pd.Series:
        
        column = column.copy(deep=True)

        # Extract values metadata.
        missing_mask = column.isna()
        number_missing_values = missing_mask.sum()
        column_range = column.max() - column.min()

        # Shift data
        column_shift = column.min() - column.min() * proportion_below

        # Create the "jitter" (noise) to be added around the points.
        np.random.seed(seed)
        column_jitter = (np.random.rand(number_missing_values) - 2) * column_range * jitter

        # Save new dummy data.
        column[missing_mask] = column_shift + column_jitter

        return column
    
    

    ################################### Plotting functions with matplotlib and seaborn ############################################
    
    #metodo para visualización de cantidad de valores faltantes por columna
    def missing_variable_plot(self):
        df = self._obj.missing.missing_variable_summary().sort_values("n_missing")

        plot_range = range(1, len(df.index) + 1)

        plt.hlines(y=plot_range, xmin=0, xmax=df.n_missing, color="black")

        plt.plot(df.n_missing, plot_range, "o", color="black")

        plt.yticks(plot_range, df.variable)

        plt.grid(axis="y")

        plt.xlabel("Number missing")
        plt.ylabel("Variable")

    #metodo para visualización agrupada de cantidad de valores faltantes por filas
    def missing_case_plot(self):

        df = self._obj.missing.missing_case_summary()

        sns.displot(data=df, x="n_missing", binwidth=1, color="black")

        plt.grid(axis="x")
        plt.xlabel("Number of missings in case")
        plt.ylabel("Number of cases")

    #metodo para visualización por intervalos de filas para visualizar valores faltantes en una columna especifica
    def missing_variable_span_plot(
        self, variable: str, span_every: int, rot: int = 0, figsize=None
    ):

        (
            self._obj.missing.missing_variable_span(
                variable=variable, span_every=span_every
            ).plot.bar(
                x="span_counter",
                y=["pct_missing", "pct_complete"],
                stacked=True,
                width=1,
                color=["black", "lightgray"],
                rot=rot,
                figsize=figsize,
            )
        )

        plt.xlabel("Span number")
        plt.ylabel("Percentage missing")
        plt.legend(["Missing", "Present"])
        plt.title(
            f"Percentage of missing values\nOver a repeating span of { span_every } ",
            loc="left",
        )
        plt.grid(False)
        plt.margins(0)
        plt.tight_layout(pad=0)

    #metodo para visualizar si los valores faltantes provienen de filas connjuntas
    def missing_upsetplot(self, variables: list[str] = None, **kwargs):

        if variables is None:
            variables = self._obj.columns.tolist()

        return (
            self._obj.isna()
            .value_counts(variables)
            .pipe(lambda df: upsetplot.plot(df, **kwargs))
        )

    # metodo para visualizar scatter plot para visualizar donde se aglomeran nuestros valores imputados
    def scatter_imputation_plot(
        self, x, y, imputation_suffix="_imp", show_marginal=False, **kwargs
    ):

        x_imputed = f"{ x }{ imputation_suffix }"
        y_imputed = f"{ y }{ imputation_suffix }"

        plot_func = sns.scatterplot if not show_marginal else sns.jointplot

        return (
            self._obj[[x, y, x_imputed, y_imputed]]
            .assign(is_imputed=lambda df: df[x_imputed] | df[y_imputed])
            .pipe(lambda df: (plot_func(data=df, x=x, y=y, hue="is_imputed", **kwargs)))
        )

    # Metodo para visualizar un grafico de mosaico con relacion a 2 variables categoricas recpecto a una tercera variable por la cual vamos a preguntar si nos faltan o no nuestros valores 
    def missing_mosaic_plot(
        self,
        target_var: str,
        x_categorical_var: str,
        y_categorical_var: str,
        ax = None
    ):
        return (
            self._obj
            .assign(
                **{target_var: lambda df: df.weight.isna().replace([True, False], ["NA", "!NA"])}
            )
            .groupby(
                [x_categorical_var, y_categorical_var, target_var],
                dropna=False,
                as_index=True,
            )
            .size()
            .pipe(
                lambda df: mosaic(
                    data=df,
                    properties=lambda key: {"color": "r" if "NA" in key else "gray"},
                    ax=ax,
                    horizontal=True,
                    axes_label=True,
                    title="",
                    labelizer=lambda key: "",
                )
            )
        )

    #metodo para visualizar valores NA y !NA desde una tabla a la que se relleno con dummies
    def missing_scatterplot_with_dummies(
        self,
        columns_NA1,
        columns_NA2,
        x,
        y
    ):
        return (
            self._obj
            .select_dtypes(
                exclude="category"
            )
            .pipe(
                lambda df: df[df.columns[df.isna().any()]]
            )
            .missing.bind_shadow_matrix(true_string=True, false_string=False)
            .apply(
                lambda column: column if "_NA" in column.name else self.column_fill_with_dummies(column, proportion_below=0.05, jitter=0.075)
            )
            .assign(
                nullity=lambda df: df[columns_NA1] | df[columns_NA2]
            )
            .pipe(
            lambda df: (
                sns.scatterplot(
                    data=df,
                    x=x,
                    y=y,
                    hue="nullity"
                    )
                )
            )
        )

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=bdf84ff9-f66c-44c7-a67c-ca1115a2b683' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>